In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

# Define global parameters
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

In [49]:
import os

dir_to_find = "tomato_dataset"
found_path = None

# Search for the directory starting from one level up '..'
# This is often needed if your notebook is in a subfolder like 'notebooks/'
for root, dirs, files in os.walk('..'):
    if dir_to_find in dirs:
        found_path = os.path.join(root, dir_to_find)
        break

if found_path:
    print(f"✅ Directory found! Use this path:\n{found_path}")
else:
    print(f"❌ Directory not found. Please check your folder structure.")

✅ Directory found! Use this path:
../data/tomato_dataset


### Tomato Model

In [ ]:
TOMATO_TRAIN_DIR = "../data/tomato_dataset/train/"
TOMATO_VAL_DIR = "../data/tomato_dataset/validation/"

tomato_train_ds = tf.keras.utils.image_dataset_from_directory(
    TOMATO_TRAIN_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

tomato_val_ds = tf.keras.utils.image_dataset_from_directory(
    TOMATO_VAL_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

tomato_class_names = tomato_train_ds.class_names
print("Classes found:", tomato_class_names)

base_model_tomato = tf.keras.applications.MobileNetV3Small(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model_tomato.trainable = False

inputs_tomato = keras.Input(shape=(224, 224, 3))
x = tf.keras.applications.mobilenet_v3.preprocess_input(inputs_tomato)
x = base_model_tomato(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs_tomato = layers.Dense(len(tomato_class_names), activation='softmax')(x)
tomato_model = keras.Model(inputs_tomato, outputs_tomato)

# Compile with initial learning rate
initial_learning_rate = 0.001
tomato_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# Early stopping callback
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

# Learning rate scheduler
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

# Train with callbacks
print("\n--- Training the Tomato Disease Model ---")
history_tomato = tomato_model.fit(
    tomato_train_ds,
    validation_data=tomato_val_ds,
    epochs=100,
    callbacks=[early_stopping, lr_scheduler]
)

# Save model
tomato_model.save("/Users/rudrasawant/Downloads/Disease Prediction/models/tomato_disease_model.h5")
print("\n✅ Tomato model saved successfully!")

Found 0 files belonging to 0 classes.
Using 0 files for training.


ValueError: No images found in directory ../data/tomato_dataset/train/Healthy/. Allowed formats: ('.bmp', '.gif', '.jpeg', '.jpg', '.png')

In [ ]:
RICE_DATA_DIR = "../data/rice_dataset"

# 1. Create the data loaders for the rice dataset
rice_train_ds = tf.keras.utils.image_dataset_from_directory(
    RICE_DATA_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

rice_val_ds = tf.keras.utils.image_dataset_from_directory(
    RICE_DATA_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

rice_class_names = rice_train_ds.class_names
print("Rice classes:", rice_class_names)

# 2. Build the model architecture
base_model_rice = tf.keras.applications.MobileNetV3Small(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model_rice.trainable = False

inputs_rice = keras.Input(shape=(224, 224, 3))
x = tf.keras.applications.mobilenet_v3.preprocess_input(inputs_rice)
x = base_model_rice(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
outputs_rice = layers.Dense(len(rice_class_names), activation='softmax')(x)
rice_model = keras.Model(inputs_rice, outputs_rice)

# 3. Compile the model
rice_model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

# 4. Train the model on rice data only
print("\n--- Training the Rice Disease Model ---")
history_rice = rice_model.fit(
    rice_train_ds,
    validation_data=rice_val_ds,
    epochs=10
)

# 5. Save the specialized model
rice_model.save("rice_disease_model.keras")
print("\n✅ Rice model saved successfully!")

In [ ]:
# 1. Create a "model registry" to map crop names to their saved model files
model_registry = {
    "Tomato": {"path": "tomato_disease_model.keras", "classes": tomato_class_names},
    "Rice": {"path": "rice_disease_model.keras", "classes": rice_class_names}
}

# 2. Create a master prediction function
def predict_disease(crop_name, image_path):
    if crop_name not in model_registry:
        return f"Error: No model found for crop '{crop_name}'."
    
    # Load the correct model and class names from the registry
    model_info = model_registry[crop_name]
    model = keras.models.load_model(model_info["path"])
    class_names = model_info["classes"]
    
    # Load and preprocess the image
    img = keras.utils.load_img(image_path, target_size=IMAGE_SIZE)
    img_array = keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    # Make a prediction
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    
    # Return the predicted class and confidence
    predicted_class = class_names[np.argmax(score)]
    confidence = 100 * np.max(score)
    
    return f"This is a {crop_name} with {predicted_class} ({confidence:.2f}% confidence)."

In [ ]:
'''
--- DEMONSTRATION ---
Assume you have a test image of a tomato
(You would replace this with a real image path)
test_tomato_image = "path/to/your/tomato.jpg" 
prediction = predict_disease("Tomato", test_tomato_image)
print(prediction)

Assume you have a test image of rice
test_rice_image = "path/to/your/rice.jpg"
prediction = predict_disease("Rice", test_rice_image)
print(prediction)
'''

In [ ]:
# --- This would be a new cell in your notebook ---
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Let's evaluate the tomato model as an example
TOMATO_TEST_DIR = "datasets/tomato_dataset/test/"

# Create a test dataset
tomato_test_ds = tf.keras.utils.image_dataset_from_directory(
    TOMATO_TEST_DIR,
    shuffle=False, # Important: keep order for confusion matrix
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE
)

In [ ]:
# Evaluate the model
print("--- Evaluating Tomato Model ---")
loss, accuracy = tomato_model.evaluate(tomato_test_ds)
print(f"Test Accuracy: {accuracy*100:.2f}%")

# Create a confusion matrix
y_true = []
y_pred_probs = []
for images, labels in tomato_test_ds:
    y_true.extend(labels.numpy())
    y_pred_probs.extend(tomato_model.predict(images))

y_pred = np.argmax(y_pred_probs, axis=1)
cm = confusion_matrix(y_true, y_pred)
class_names = tomato_test_ds.class_names

In [ ]:
# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix for Tomato Model')
plt.show()

## Model Refinement and Improvement
How you'll likely want to improve your models.

Fine-Tuning: Unfreeze the last few layers of the pre-trained base_model and retrain the whole model with a very low learning rate. This can significantly boost accuracy.

Get More Data: If your model is confusing two diseases, it means you need more varied image examples of those specific conditions. More high-quality data is the single best way to improve performance.
